In [47]:
%ls

accuracy.png        chainer book.ipynb  loss.png
cg.dot              log                 snapshot_iter_600


In [48]:
!ls

accuracy.png       chainer book.ipynb loss.png
cg.dot             log                snapshot_iter_600


In [ ]:
#https://play.chainer.org/book

# Variable

In [1]:
from chainer import Variable
import numpy as np


x_data = np.array([5], dtype=np.float32)
x = Variable(x_data)
y = x ** 2 - 2 * x + 1
print(y.data)

z_data = np.array([[2, 3, 4], [5, 6, 7]], dtype=np.float32)
z = Variable(z_data)
print(z[:, 1].data)  # [[3], [6]]


/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[16.]
[3. 6.]


# backward

In [30]:
from chainer import Variable
import numpy as np

x_data = np.array([5], dtype=np.float32)
x = Variable(x_data)
y = x ** 2 - 2 * x + 1
y.backward()

z = Variable(np.array([10, 20], dtype=np.float32))
zz = 2 * z
zz.grad = np.array([0.1, -0.1], dtype=np.float32)
zz.backward()

print(x.grad)
print(z.grad)

# print(y.data)


[8.]
[ 0.2 -0.2]


In [31]:
#1回目
y.backward()
zz.backward()
print(x.grad)
print(z.grad)

[16.]
[ 0.4 -0.4]


In [32]:
#2回目
y.backward()
zz.backward()
print(x.grad)
print(z.grad)
#gradが変わっていっている

[24.]
[ 0.6 -0.6]


In [33]:
#unchain
y.unchain_backward()
zz.unchain_backward()
print(x.grad)
print(z.grad)

[24.]
[ 0.6 -0.6]


In [34]:
#3回目
y.backward()
zz.backward()
print(x.grad)
print(z.grad)

[24.]
[ 0.6 -0.6]


In [35]:
#４回目
y.backward()
zz.backward()
print(x.grad)
print(z.grad)
#chainを切った場所以前の値が変わらない。

[24.]
[ 0.6 -0.6]


In [46]:
print(y.data)

[16.]


# Optimization

In [11]:
from chainer import Chain
from chainer import functions as F
from chainer import links as L
from chainer import optimizers
from chainer import Variable
import numpy as np


class Linear(Chain):
    def __init__(self):
        super(Linear, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(1, 1)

    def __call__(self, x):
        return self.l1(x)


def f(x):
    return 5. * x + 10


x = np.linspace(-10, 10, num=11)
y = f(x) + 5. * np.random.randn(11)

model = Linear()
#cleargradsしないとnanになったりするので注意！！！
model.cleargrads()
opt = optimizers.SGD(lr=0.001)
opt.setup(model)
for epoch in range(2):
    perm = np.random.permutation(len(x))
    for i in range(len(x)):
        x_i = Variable(np.array([[x[perm[i]]]], dtype=np.float32))
        y_i = Variable(np.array([[y[perm[i]]]], dtype=np.float32))
        print(x_i[0])
        print(y_i[0])
        _y = model(x_i)
        loss = F.mean_squared_error(_y, y_i)
        loss.backward()
        opt.update()
        print("loss:", loss)

variable([2.])
variable([17.676245])
loss: variable(251.04852)
variable([10.])
variable([54.024258])
loss: variable(1953.6505)
variable([-2.])
variable([1.7146585])
loss: variable(29.334475)
variable([6.])
variable([43.24403])
loss: variable(668.08203)
variable([-10.])
variable([-39.305027])
loss: variable(1.2363309)
variable([-8.])
variable([-28.850517])
loss: variable(166.77892)
variable([8.])
variable([42.773113])
loss: variable(73.25318)
variable([-4.])
variable([-9.691114])
loss: variable(322.9339)
variable([-6.])
variable([-23.670183])
loss: variable(505.24258)
variable([4.])
variable([25.64906])
loss: variable(87.54749)
variable([0.])
variable([10.642519])
loss: variable(77.94192)
variable([-2.])
variable([1.7146585])
loss: variable(319.61743)
variable([10.])
variable([54.024258])
loss: variable(1825.6501)
variable([-6.])
variable([-23.670183])
loss: variable(732.0652)
variable([2.])
variable([17.676245])
loss: variable(1.6032312)
variable([0.])
variable([10.642519])
loss: varia

# loss

例えばsoftmax_cross_entropyはdtypeもfとiになってるので注意。

In [54]:
x = np.array([[-1, 0, 1, 2], [2, 0, 1, -1]]).astype('f')
t = np.array([3, 0]).astype('i')
y = F.softmax_cross_entropy(x, t)
log_softmax = -F.log_softmax(x)
expected_loss = np.mean([log_softmax[row, column].data for row, column in enumerate(t)])
y.array == expected_loss

True

# manually assigned Variable

In [ ]:
#https://github.com/chainer/chainer/issues/3868
#https://twitter.com/im132nd/status/871751800847609857

In [41]:
from chainer import Variable
import numpy as np


x_data = np.array([5], dtype=np.float32)
x = Variable(x_data)
y = x ** 2 - 2 * x + 1
print(y.data)
print(y._grad_var)
print(y._node)
print(x.node.dtype)
print(x.node.data.dtype)

z_data = np.array([[2, 3, 4], [5, 6, 7]], dtype=np.float32)
z = Variable(z_data)
print(z[:, 1].data)  # [[3], [6]]


[16.]
None
float32
float32
[3. 6.]


In [42]:
import chainer
import numpy
from chainer import functions as F

x = chainer.Variable(numpy.array([3, 2], dtype=numpy.float32))
y = F.sin(x)
x.data = numpy.array([4, 2], dtype=numpy.float64)

print(x.node.dtype)
print(x.node.data.dtype)

float64
float64


# shapeとdtypeを確認する

In [37]:
# -*- coding: utf-8 -*-
import chainer
from chainer import functions as F
from chainer import links as L
from chainer import Variable
from chainer import optimizers
import numpy as np

class MLP(chainer.Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)    # n_units -> n_out
    def __call__(self, x, t):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        print("==model call==")
        print(y.shape)
        print(y.ndim)
        print(t.shape)
        print(t.ndim)
#         print(y.data)
        print(y._grad_var)
        return F.softmax_cross_entropy(y, t)
    
    def predict(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return F.softmax(self.l3(h2))

model = MLP(5, 2)
model.cleargrads()
optimizer = optimizers.SGD(lr=0.001)
optimizer.setup(model)
x = Variable(np.ones((3, 5), dtype=np.float32))
t = Variable(np.ones((3), dtype=np.int32))
print("==input==")
print(x)
print(t)
loss = model(x, t)
print("loss:", loss.data)
print("x grad:", x.grad)

==input==
variable([[1. 1. 1. 1. 1.]
          [1. 1. 1. 1. 1.]
          [1. 1. 1. 1. 1.]])
variable([1 1 1])
==model call==
(3, 2)
2
(3,)
1
None
loss: 0.6528654
x grad: None


# gradの動きを確認する

In [38]:
# -*- coding: utf-8 -*-
import chainer
from chainer import functions as F
from chainer import links as L
from chainer import Variable
from chainer import optimizers
import numpy as np

class MLP(chainer.Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)    # n_units -> n_out
    def __call__(self, x, t):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return F.softmax_cross_entropy(y, t)
    
    def predict(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return F.softmax(self.l3(h2))

model = MLP(5, 2)
model.cleargrads()
optimizer = optimizers.SGD(lr=0.001)
optimizer.setup(model)
x = Variable(np.ones((3, 5), dtype=np.float32))
t = Variable(np.ones((3), dtype=np.int32))
print("==input==")
print(x)
print(t)
loss = model(x, t)
print("loss:", loss.data)
print("x grad:", x.grad)

==input==
variable([[1. 1. 1. 1. 1.]
          [1. 1. 1. 1. 1.]
          [1. 1. 1. 1. 1.]])
variable([1 1 1])
loss: 1.4252744
x grad: None


In [39]:
print("==backward==")
loss.backward()
loss = model(x, t)
print("loss:", loss.data)
print("x grad:", x.grad)

==backward==
loss: 1.4252744
x grad: [[ 0.14618626 -0.0203914   0.12454231 -0.00858422  0.0494754 ]
 [ 0.14618626 -0.0203914   0.12454231 -0.00858422  0.0494754 ]
 [ 0.14618626 -0.0203914   0.12454231 -0.00858422  0.0494754 ]]


In [40]:
print("==backward==")
loss.backward()
loss = model(x, t)

print("loss:", loss.data)
print("x grad:", x.grad)

==backward==
loss: 1.4252744
x grad: [[ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]
 [ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]
 [ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]]


gradは変わってるが当然lossは変わらない。

# 重み係数を更新する

In [41]:
optimizer.update()
loss = model(x, t)
print("loss:", loss.data)
print("x grad:", x.grad)

loss: 1.4081409
x grad: [[ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]
 [ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]
 [ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]]


In [42]:
optimizer.update()
loss = model(x, t)
print("loss:", loss.data)
print("x grad:", x.grad)

loss: 1.3912008
x grad: [[ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]
 [ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]
 [ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]]


In [43]:
optimizer.update()
loss = model(x, t)
print("loss:", loss.data)
print("x grad:", x.grad)
#gradは変わってないが、係数は更新されている

loss: 1.3744533
x grad: [[ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]
 [ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]
 [ 0.29237252 -0.0407828   0.24908462 -0.01716844  0.0989508 ]]


In [44]:
optimizer.update(model, x, t)
loss = model(x, t)
print("loss:", loss.data)
print("x grad:", x.grad)
#lossとgradと係数に変化がある。
#optimizer.update(,,)を使うと内部でbackwardもやる

loss: 1.3664272
x grad: [[ 0.4311268  -0.06172036  0.3664877  -0.02553374  0.14538176]
 [ 0.4311268  -0.06172036  0.3664877  -0.02553374  0.14538176]
 [ 0.4311268  -0.06172036  0.3664877  -0.02553374  0.14538176]]


# load_irisを使ってみる

In [17]:
#データセットを変える

In [48]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
X = X.astype(np.float32)

Y = iris.target
Y = Y.flatten().astype(np.int32)

print(X.shape)
print(Y.shape)
print(X[0])
print(Y) #4つの特徴量で3つのクラス

(150, 4)
(150,)
[5.1 3.5 1.4 0.2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [19]:
X = Variable(X)
Y = Variable(Y)

In [33]:
X[0]
x[0].shape

(5,)

In [58]:
# -*- coding: utf-8 -*-
import chainer
from chainer import functions as F
from chainer import links as L
from chainer import Variable
import numpy as np

class MLP(chainer.Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)    # n_units -> n_out
    def __call__(self, x, t):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
#         print(y)
#         print(y.shape)
        return F.softmax_cross_entropy(y, t)
    
    def predict(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return F.softmax(self.l3(h2))

model = MLP(10, 3)
model.cleargrads()
print("==input==")
loss = model(X, Y)
print("loss:", loss.data)
print("x grad:", x.grad)

==input==
loss: 1.8300899
x grad: [[ 0.00861229  0.00291787 -0.00866771  0.00900884 -0.00604716]
 [ 0.00861229  0.00291787 -0.00866771  0.00900884 -0.00604716]
 [ 0.00861229  0.00291787 -0.00866771  0.00900884 -0.00604716]]


In [68]:
print("predict:", model.predict(np.array([[1.1,1.5,1.4,0.2]], dtype=np.float32)))
print("predict max:", model.predict(np.array([[1.1,1.5,1.4,0.2]], dtype=np.float32)).data.argmax(axis=1))

predict: variable([[0.27109733 0.25443485 0.4744678 ]])
predict max: [2]


In [69]:
print("predict:", model.predict(np.array([[5.1,3.5,1.4,0.2]], dtype=np.float32)))
print("predict max:", model.predict(np.array([[5.1,3.5,1.4,0.2]], dtype=np.float32)).data.argmax(axis=1))

predict: variable([[0.22469056 0.03719145 0.73811793]])
predict max: [2]


In [21]:
print("==backward==")
#retain_grad=True
loss.backward(retain_grad=False)
loss = model(X, Y)
print("loss:", loss.data)
print("x grad:", x.grad)

==backward==
loss: 1.2160245
x grad: [[ 0.00861229  0.00291787 -0.00866771  0.00900884 -0.00604716]
 [ 0.00861229  0.00291787 -0.00866771  0.00900884 -0.00604716]
 [ 0.00861229  0.00291787 -0.00866771  0.00900884 -0.00604716]]


In [22]:
optimizer = optimizers.SGD(lr=0.00001)
optimizer.setup(model)
optimizer.update()

In [23]:
loss = model(X, Y)
print("loss:", loss.data)

loss: 1.216


In [24]:
model.l1.W

variable W([[-0.3016114 ,  0.18870594,  0.1085281 ,  0.118176  ],
            [-0.37841696, -0.00885908,  0.56859565,  0.25816032]])

In [25]:
model.l1.b

variable b([0.000000e+00, 1.739527e-06])

In [93]:
model.cleargrads()
for i in range(200):
    loss = model(X, Y)
    loss.backward(retain_grad=False)
    optimizer = optimizers.SGD(lr=0.001)
    optimizer.setup(model)
    optimizer.update()
print("loss:", loss)

loss: variable(0.06108605)


In [94]:
print("predict:", model.predict(np.array([[0.1,0.5,0.4,5.2]], dtype=np.float32)))
print("predict max:", model.predict(np.array([[0.1,0.5,0.4,5.2]], dtype=np.float32)).data.argmax(axis=1))

predict: variable([[3.3553846e-08 3.6883622e-04 9.9963117e-01]])
predict max: [2]


In [95]:
print("predict:", model.predict(np.array([[5.1,3.5,1.4,0.2]], dtype=np.float32)))
print("predict max:", model.predict(np.array([[5.1,3.5,1.4,0.2]], dtype=np.float32)).data.argmax(axis=1))

predict: variable([[9.9980384e-01 1.9615908e-04 1.3166370e-21]])
predict max: [0]


# Trainer

In [16]:
#!/usr/bin/env python

from __future__ import print_function

import argparse

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions
from chainer import report, training, Chain, datasets, iterators, optimizers
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.datasets import tuple_dataset



# Network definition
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)


def main():
    unit = 1000
    batchsize = 100
    gpu = -1
    epoch = 1
    frequency = -1
    out = "./"
    plot = True
    model = L.Classifier(MLP(unit, 10))
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)
    train, test = chainer.datasets.get_mnist()
    print(train[0][0].shape)
    print(train[0][1])    
    train_iter = chainer.iterators.SerialIterator(train, batchsize)
    test_iter = chainer.iterators.SerialIterator(test, batchsize,
                                                 repeat=False, shuffle=False)
    updater = training.StandardUpdater(
        train_iter, optimizer, device=gpu)
    trainer = training.Trainer(updater, (epoch, 'epoch'), out=out)
    trainer.extend(extensions.Evaluator(test_iter, model, device=gpu))
    trainer.extend(extensions.dump_graph('main/loss'))
    frequency = epoch if frequency == -1 else max(1, frequency)
    trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))
    trainer.extend(extensions.LogReport())
    if plot and extensions.PlotReport.available():
        trainer.extend(
            extensions.PlotReport(['main/loss', 'validation/main/loss'],
                                  'epoch', file_name='loss.png'))
        trainer.extend(
            extensions.PlotReport(
                ['main/accuracy', 'validation/main/accuracy'],
                'epoch', file_name='accuracy.png'))
    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
    trainer.extend(extensions.ProgressBar())
    trainer.run()

if __name__ == '__main__':
    main()

(784,)
5
epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [########..........................................] 16.67%
this epoch [########..........................................] 16.67%
       100 iter, 0 epoch / 1 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [################..................................] 33.33%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 1 epochs
     20.16 iters/sec. Estimated time to finish: 0:00:19.841736.
     total [#########################.........................] 50.00%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 1 epochs
    20.621 iters/sec. Estimated time to finish: 0:00:14.548076.
     total [#################################.................] 66.67%
this epoch [#################################.................] 66.67%
       400 iter, 0 epoch / 1 e

In [130]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
X = X.astype(np.float32)
X = X.reshape(150, 4, 1)

Y = iris.target
Y = Y.flatten().astype(np.int32)
Y = Y.reshape(150, 1)


print(X.shape)
print(Y.shape)
X = Variable(X)
Y = Variable(Y)

train_data = [[x, y] for x, y in zip(X, Y)]
train_data[0]

(150, 4, 1)
(150, 1)


[variable([[5.1],
           [3.5],
           [1.4],
           [0.2]]), variable([0])]

In [132]:
import chainer
from chainer import datasets
from chainer import functions as F
from chainer import links as L
from chainer import training
from chainer.training import extensions

class MLP(chainer.Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)    # n_units -> n_out
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

batchsize = 1
# # train, test = datasets.get_mnist()
# train = datasets.SubDataset(train, 0, 100)
# test = datasets.SubDataset(test, 0, 100)
train_iter = chainer.iterators.SerialIterator(train_data, batchsize)
test_iter = chainer.iterators.SerialIterator(train_data, batchsize,
                                             repeat=False, shuffle=False)
model = L.Classifier(MLP(10, 3))
opt = chainer.optimizers.Adam()
opt.setup(model)
epoch = 1
# Set up a trainer
updater = training.StandardUpdater(train_iter, opt, device=-1)
trainer = training.Trainer(updater, (epoch, 'epoch'), out='./')
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy']))
resume = False
if resume:
    chainer.serializers.load_npz(resume, trainer)
trainer.run()


Exception in main training loop: __call__() missing 1 required positional argument: 'x'
Traceback (most recent call last):
  File "/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/chainer/training/trainer.py", line 299, in run
    update()
  File "/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/chainer/training/updater.py", line 223, in update
    self.update_core()
  File "/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/chainer/training/updater.py", line 238, in update_core
    optimizer.update(loss_func, in_arrays)
  File "/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/chainer/optimizer.py", line 541, in update
    loss = lossfun(*args, **kwds)
  File "/Users/miyamoto/.pyenv/versions/miniconda3-4.1.11/envs/py35/lib/python3.5/site-packages/chainer/links/model/classifier.py", line 114, in __call__
    self.y = self.predictor(*ar

TypeError: __call__() missing 1 required positional argument: 'x'